In [1]:
import findspark
findspark.init()

In [2]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType
from pyspark import SparkFiles

# create spark session
spark: SparkSession = SparkSession.builder.appName("SparkSQL").config("spark.driver.memory", "2g").getOrCreate()

In [3]:
companies = ['AEP', 'COMED, DAYTON', 'DEOK', 'DOM', 'DUQ', 'EKPC', 'FE', 'NI', 'PJME', 'PJMW']
for company in companies:
    base_PJM_url = f'C:/Users/lukej/OneDrive/Documents/GitHub/PJM-Energy-Consumption--Stock-Data/Resources/PJM/{company}_hourly.csv'
    spark.sparkContext.addFile(base_PJM_url)
    company = spark.read.csv(SparkFiles.get(f'{company}_hourly.csv"'), header=True, inferSchema=True)


Py4JJavaError: An error occurred while calling o27.addFile.
: org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "C"
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3443)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.SparkContext.addFile(SparkContext.scala:1754)
	at org.apache.spark.SparkContext.addFile(SparkContext.scala:1733)
	at java.base/jdk.internal.reflect.DirectMethodHandleAccessor.invoke(DirectMethodHandleAccessor.java:103)
	at java.base/java.lang.reflect.Method.invoke(Method.java:580)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.ja

In [54]:
# pandas way if I can't pyspark to work
import pandas as pd

companies = ['AEP', 'COMED', 'DAYTON', 'DEOK', 'DOM', 'DUQ', 'EKPC', 'FE', 'NI', 'PJME', 'PJMW']

# dictionary comprehension of all companies
dfs = {company: pd.read_csv(f"Resources/PJM/{company}_hourly.csv").set_index('Datetime') for company in companies}

# showing example
dfs['FE']

,FE_MW
Datetime,
2011-12-31 01:00:00,6222.0
2011-12-31 02:00:00,5973.0
2011-12-31 03:00:00,5778.0
2011-12-31 04:00:00,5707.0
2011-12-31 05:00:00,5691.0
...,...
2018-01-01 20:00:00,9378.0
2018-01-01 21:00:00,9255.0
2018-01-01 22:00:00,9044.0


In [57]:
# made shortcut for index


dfs['FE'].index

Index(['2011-12-31 01:00:00', '2011-12-31 02:00:00', '2011-12-31 03:00:00',
       '2011-12-31 04:00:00', '2011-12-31 05:00:00', '2011-12-31 06:00:00',
       '2011-12-31 07:00:00', '2011-12-31 08:00:00', '2011-12-31 09:00:00',
       '2011-12-31 10:00:00',
       ...
       '2018-01-01 15:00:00', '2018-01-01 16:00:00', '2018-01-01 17:00:00',
       '2018-01-01 18:00:00', '2018-01-01 19:00:00', '2018-01-01 20:00:00',
       '2018-01-01 21:00:00', '2018-01-01 22:00:00', '2018-01-01 23:00:00',
       '2018-01-02 00:00:00'],
      dtype='object', name='Datetime', length=62874)

In [63]:
import hvplot.pandas
from bokeh.models.formatters import DatetimeTickFormatter

formatter = DatetimeTickFormatter(months='%b %Y')

company_plots = []
for company in companies:
    company_plots.append(dfs[company].hvplot.line(x = 'index', 
                                                  y = f'{company}_MW',
                                                  title = f"{company}'s energy production over time",
                                                  xformatter=formatter))
company_plots[6]

:Curve   [index]   (EKPC_MW)